# Algorithm Efficiency Mini-Project

This notebook contains implementations of several algorithms, timing experiments, and plots for execution time vs input size. It was generated automatically.

**Includes:** Fibonacci (recursive & DP), Merge Sort, Quick Sort, Insertion Sort, Bubble Sort, Selection Sort, Binary Search. Timing uses `time.perf_counter` and matplotlib for plots.

*Profiling mode:* Execution time only (no memory profiling)

In [ ]:
# Imports and utility functions
import time
import random
import math
import matplotlib.pyplot as plt
import statistics
# Note: per notebook generation rules, we will not set explicit matplotlib styles or colors.

def time_function(func, *args, repeats=3, **kwargs):
    """Run func(*args, **kwargs) `repeats` times and return average time in seconds."""
    times = []
    for _ in range(repeats):
        t0 = time.perf_counter()
        func(*args, **kwargs)
        t1 = time.perf_counter()
        times.append(t1 - t0)
    return statistics.mean(times)


## Algorithm Implementations

In [ ]:
# Fibonacci (naive recursive)
def fib_recursive(n):
    if n <= 1:
        return n
    return fib_recursive(n-1) + fib_recursive(n-2)

# Fibonacci (dynamic programming)
def fib_dp(n):
    if n <= 1:
        return n
    dp = [0]*(n+1)
    dp[0], dp[1] = 0, 1
    for i in range(2, n+1):
        dp[i] = dp[i-1] + dp[i-2]
    return dp[n]


In [ ]:
# Merge Sort
def merge_sort(arr):
    if len(arr) <= 1:
        return arr[:]
    mid = len(arr)//2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    # merge
    i = j = 0
    out = []
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            out.append(left[i]); i += 1
        else:
            out.append(right[j]); j += 1
    out.extend(left[i:])
    out.extend(right[j:])
    return out


In [ ]:
# Quick Sort (in-place-ish using list slicing to keep simple)
def quick_sort(arr):
    if len(arr) <= 1:
        return arr[:]
    pivot = arr[len(arr)//2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quick_sort(left) + middle + quick_sort(right)


In [ ]:
# Insertion Sort
def insertion_sort(arr):
    a = arr[:]
    for i in range(1, len(a)):
        key = a[i]
        j = i-1
        while j >= 0 and a[j] > key:
            a[j+1] = a[j]
            j -= 1
        a[j+1] = key
    return a

# Bubble Sort
def bubble_sort(arr):
    a = arr[:]
    n = len(a)
    for i in range(n):
        swapped = False
        for j in range(0, n-i-1):
            if a[j] > a[j+1]:
                a[j], a[j+1] = a[j+1], a[j]
                swapped = True
        if not swapped:
            break
    return a

# Selection Sort
def selection_sort(arr):
    a = arr[:]
    n = len(a)
    for i in range(n):
        min_idx = i
        for j in range(i+1, n):
            if a[j] < a[min_idx]:
                min_idx = j
        a[i], a[min_idx] = a[min_idx], a[i]
    return a


In [ ]:
# Binary Search (assumes sorted array)
def binary_search(arr, target):
    lo, hi = 0, len(arr)-1
    while lo <= hi:
        mid = (lo + hi) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            lo = mid + 1
        else:
            hi = mid - 1
    return -1


## Complexity Summary

- **Fibonacci (naive recursive):** Time O(2^n), Space O(n) recursion.
- **Fibonacci (DP):** Time O(n), Space O(n).
- **Merge Sort:** Time O(n log n), Space O(n).
- **Quick Sort:** Average O(n log n), Worst O(n^2) (depends on pivot), Space O(log n) average.
- **Insertion/Bubble/Selection Sort:** Time O(n^2) average/worst, Space O(1).
- **Binary Search:** Time O(log n), Space O(1).


## Timing Experiments

We'll run timing experiments for each algorithm. For `O(n^2)` algorithms we use smaller input sizes to keep run time reasonable. For Fibonacci recursive, we'll use very small n.

In [ ]:
# Prepare input sizes for different algorithm classes
sizes_nlogn = [100, 500, 1000, 2000, 5000, 10000]   # for merge/quick/binary
sizes_n2 = [100, 200, 400, 800, 1600, 2000]         # for insertion/bubble/selection
sizes_fib = [5, 10, 15, 20, 25, 30]                # for naive recursive (stop at 30)
random_seed = 42
random.seed(random_seed)

# Helper to generate random lists
def random_list(n, low=0, high=10**6):
    return [random.randint(low, high) for _ in range(n)]


In [ ]:
# Run timings and collect results
results = {}

# Merge Sort & Quick Sort (n log n)
for algo_name, algo in [('merge_sort', merge_sort), ('quick_sort', quick_sort)]:
    times = []
    for n in sizes_nlogn:
        arr = random_list(n)
        t = time_function(algo, arr, repeats=3)
        times.append(t)
        print(f"{algo_name} n={n} time={t:.6f}s")
    results[algo_name] = (sizes_nlogn, times)

# Insertion, Bubble, Selection (n^2)
for algo_name, algo in [('insertion_sort', insertion_sort), ('bubble_sort', bubble_sort), ('selection_sort', selection_sort)]:
    times = []
    for n in sizes_n2:
        arr = random_list(n)
        t = time_function(algo, arr, repeats=3)
        times.append(t)
        print(f"{algo_name} n={n} time={t:.6f}s")
    results[algo_name] = (sizes_n2, times)

# Binary Search (log n) - measure searching in sorted list
algo_name = 'binary_search'
times = []
for n in sizes_nlogn:
    arr = sorted(random_list(n))
    # pick existing target to keep consistent
    target = arr[n//2] if n>0 else None
    t = time_function(binary_search, arr, target, repeats=50)  # many repeats for tiny time
    times.append(t)
    print(f"{algo_name} n={n} time={t:.9f}s")
results[algo_name] = (sizes_nlogn, times)

# Fibonacci DP (linear)
algo_name = 'fib_dp'
times = []
for n in [1000, 2000, 5000, 10000]:
    t = time_function(fib_dp, n, repeats=3)
    times.append(t)
    print(f"{algo_name} n={n} time={t:.6f}s")
results[algo_name] = ([1000,2000,5000,10000], times)

# Fibonacci recursive (exponential) - small sizes only
algo_name = 'fib_recursive'
times = []
for n in sizes_fib:
    if n > 30:
        break
    t = time_function(fib_recursive, n, repeats=3)
    times.append(t)
    print(f"{algo_name} n={n} time={t:.6f}s")
results[algo_name] = (sizes_fib[:len(times)], times)

# Save results for plotting
import pickle
with open('/mnt/data/timing_results.pkl', 'wb') as f:
    pickle.dump(results, f)

print('\nTiming experiments completed. Results saved to /mnt/data/timing_results.pkl')

### Plotting results

We'll create one combined figure with subplots for categories: n log n algorithms, n^2 algorithms, and special cases (binary search, fibonacci).

In [ ]:
# Load results and plot
import pickle
with open('/mnt/data/timing_results.pkl','rb') as f:
    results = pickle.load(f)

fig, axs = plt.subplots(2, 2, figsize=(14,10))
axs = axs.flatten()

# Plot n log n algorithms
for name in ['merge_sort', 'quick_sort']:
    ns, ts = results[name]
    axs[0].plot(ns, ts, marker='o', label=name)
axs[0].set_title('O(n log n) algorithms')
axs[0].set_xlabel('n')
axs[0].set_ylabel('time (s)')
axs[0].legend()

# Plot n^2 algorithms
for name in ['insertion_sort', 'bubble_sort', 'selection_sort']:
    ns, ts = results[name]
    axs[1].plot(ns, ts, marker='o', label=name)
axs[1].set_title('O(n^2) algorithms')
axs[1].set_xlabel('n')
axs[1].set_ylabel('time (s)')
axs[1].legend()

# Binary search
ns, ts = results['binary_search']
axs[2].plot(ns, ts, marker='o')
axs[2].set_title('Binary Search O(log n)')
axs[2].set_xlabel('n')
axs[2].set_ylabel('time (s)')

# Fibonacci
ns, ts = results['fib_recursive']
axs[3].plot(ns, ts, marker='o', label='fib_recursive')
if 'fib_dp' in results:
    ns_dp, ts_dp = results['fib_dp']
    axs[3].plot(ns_dp, ts_dp, marker='o', label='fib_dp')
axs[3].set_title('Fibonacci: recursive vs DP (different n scales)')
axs[3].set_xlabel('n')
axs[3].set_ylabel('time (s)')
axs[3].legend()

plt.tight_layout()
plot_path = '/mnt/data/execution_time_plots.png'
plt.savefig(plot_path)
plt.show()

print(f"Plot saved to {plot_path}")

## Summary Table and Reflections

- The plots above show growth patterns matching theoretical complexities: O(n log n) algorithms scale much better than O(n^2) algorithms.
- Naive recursive Fibonacci grows exponentially and becomes infeasible beyond n ~ 30; DP version runs in linear time and is practical for large n.
- Binary search time per query is extremely small and appears nearly constant in wall-clock timing because individual calls are short; repeated measurements show more stable averages.

**Next steps for submission:**
- Move this notebook and `execution_time_plots.png` to your GitHub repo.
- Add README.md and requirements.txt describing how to run the notebook.
